In [ ]:
from utils.bech32m import convertbits, bech32_encode, Encoding
from utils.key import ECKey, ECPubKey, generate_bip340_key_pair
from utils.bip158 import gcs_match_any

from bip32 import BIP32
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException

import hashlib
def sha256(s):
    if type(s) != bytes:
        s = s.get_bytes()
    return hashlib.new('sha256', s).digest()

G = ECKey().set(1).get_pubkey()

# Silent Payments

Using a new address for each Bitcoin transaction is a crucial aspect of maintaining privacy. This often requires a secure interaction between sender and receiver so that the receiver can hand out a fresh address, a batch of fresh address, or a way for the send to generate addresses, such as an xpub.

However, interaction is often infeasible and in many cases undesirable. To solve for this, various protocols have been proposed which use a static payment address and notifications, sent via the blockchain[footnote]. These protocols eliminate the need for interaction, but at the expense of increased costs for one-time payments and a noticeable footprint in the blockchain, potentially revealing metadata about the sender and receiver. Notification schemes also allow the receiver to link all payments from the same sender, compromising sender privacy.

This proposal aims to address the limitations of these current approaches by presenting a solution that eliminates the need for interaction, eliminates the need for notifications, and protects both sender and receiver privacy.

## Goals

We aim to present a transaction protocol which satisifies the following properties:

* No increase in the size or cost of transactions
* Resulting transactions blend in with other bitcoin transactions and can't be distinguished
* Transactions can't be linked to a silent payment address by an outside observer
* No sender-receiver interaction required
* No linking of multiple payments to the same sender
* Each silent payment goes to a unique address, avoiding accidental reuse
* Supports payment purpose labeling
* Uses existing seed phrase or descriptor methods for backup and recovery
* Separates scanning and spending responsibilities
* Compatible with other spending protocols, such as CoinJoin
* Light client/SPV wallet support
* Protocol is upgradeable


## What this workshop will attempt to cover

* Generating a silent payment address
* Scanning for silent payments
* Sync the wallet using BIP158 block filters
* BONUS:
  * Add label support
  * Scan for multiple outputs
  * Run on signet
  
## What this workshop won't cover

* Sending to silent payment addresses

# Elliptic Curve math review

Elliptic Curve math involves scalars and points.

* A scalar is a positive integer which is smaller than the group order, and is denoted by a lower case letter (eg `a`).
* A point lies on the curve and is denoted by an upper-case letter (eg `C`) or a pair of co-ordinates (eg `(x,y)`).

In Bitcoin, key pair generation and signing is performed over the secp256k1 curve. All scalars are modulo the group order `SECP256K1_ORDER`, which is a very large number

![test](images/ec_math0.jpg)

_An overview of all operations of scalars and points over elliptic curves._

## Simple case

Bob wishes to receive payments from Alice, so he publishes his public key B as a silent payment address. Alice selects a UTXO with private key a and public key A from her wallet to fund the transaction and creates the output P for Bob in the following way:

* Let *P = HASH(a·B)·G + B*

Bob detects this payment by computing *P = HASH(b·A)·G + B*, since *a·B == b·A* (Elliptic Curve Diffie-Hellman).

In [ ]:
b, B = generate_bip340_key_pair()
a, A = generate_bip340_key_pair()

# Alice generates the output using her private key b and Bob's public key
# sha256(a * B)
t = 

# create a pubkey: T = t * G
T = 

# create the silent payment output: D = T + B
D = 

# Bob checks if D is hers using her private key and Bob's public key
t_prime = 
T_prime = 
D_prime = 

assert D == D_prime

### Sending to more than one output

<p>In the event Alice needs more than one output for Bob, she can do so in the following manner:</p>
<ul>
<li>Let <em>P<sub>0</sub> = HASH(a·B || 0)·G + B</em></li>
<li>For additional outputs:
<ul>
<li>Let <em>P<sub>i</sub> = HASH(a·B || n)·G + B</em>, where <em>n</em> starts from 1 and is incremented for each subsequent output</li>
</ul></li>
</ul>
<p>Bob detects this output the same as before by searching for <em>P<sub>0</sub> = HASH(b·A || 0)·G + B</em>. Once he detects the first output, he must:</p>
<ul>
<li>Check for <em>P<sub>1</sub> = HASH(b·A || 1)·G + B</em></li>
<li>If <em>P<sub>1</sub></em> is not found, stop</li>
<li>If <em>P<sub>1</sub></em> is found, continue to check for <em>P<sub>2</sub></em> and so on until an additional output is not found</li>
</ul>
<p>Since Bob will only perform these subsquent checks after a transaction with at least one output paying him is found, the increase to his overall scanning requirement is negligible.</p>


## Preventing address reuse

<p>If Alice were to use a different UTXO from the same public key <em>A</em> for a subsequent payment to Bob, she would end up deriving the same destination <em>P</em>. To prevent this, Alice should include a hash of the outpoint in the following manner:</p>
<ul>
<li>Let <em>outpoint_hash = HASH(txid || vout)</em></li>
<li>Let <em>P<sub>0</sub> = HASH(outpoint_hash·a·B || 0)·G + B</em></li>
</ul>

In [ ]:
# calculate the outpoint hash

txid1 = b'f4184fc596403b9d638783cf57adfe4c75c605f6356fbc91338530e9831e9e16'
txid2 = b'a1075db55d416d3ca199f55b6084e2115b9345e16c5cf302fc80e9d5fbf5d48d'
vout1 = (0).to_bytes(4, 'little')
vout2 = (1).to_bytes(4, 'little')

# sha256(txid || vout)
# TODO: these should be sorted
outpoint_hash = 

t = 
T = t * G
D = T + B

# Bob scans..

t_prime = 
T_prime = t_prime * G
D_prime = T_prime + B

assert D == D_prime

### Using all inputs

<p>In our simplified example we have been referring to Alice’s transactions as having only one input <em>A</em>, but in reality a Bitcoin transaction can have many inputs. Instead of requiring Alice to pick a particular input and requiring Bob to check each input separately, we can instead require Alice to perform the tweak with the sum of the input public keys. This significantly reduces Bob's scanning requirement, makes light client support more feasible, and protects Alice's privacy in collaborative transaction protocols such as CoinJoin.</p>
<p>Alice performs the tweak with the sum of her input private keys in the following manner:</p>
<ul>
<li>Let <em>outpoints_hash = HASH(txid<sub>1</sub> || vout<sub>1</sub> || … txid<sub>n</sub> || vout<sub>n</sub>)</em></li>
<li>Let <em>a = a<sub>0</sub> + a<sub>1</sub> … + a<sub>n</sub></em></li>
<li>Let <em>A = A<sub>0</sub> + A<sub>1</sub> … + A<sub>n</sub></em></li>
<li>Let <em>P<sub>0</sub> = HASH(outpoints_hash·a·B || 0)·G + B</em></li>
</ul>


In [ ]:
# doesn't have to be a taproot UTXO
txid1 = b'f4184fc596403b9d638783cf57adfe4c75c605f6356fbc91338530e9831e9e16'
vout1 = (0).to_bytes(4, 'little')

a1 = ECKey().generate()
A1 = a1.get_pubkey()

# taproot UTXO
txid2 = b'a1075db55d416d3ca199f55b6084e2115b9345e16c5cf302fc80e9d5fbf5d48d'
vout2 = (0).to_bytes(4, 'little')

a2, A2 = generate_bip340_key_pair()

outpoint_hash = sha256(txid1 + vout1 + txid2 + vout2)

# sha256(outputpoint_hash * Sum of Alice's input privkeys * B)
t = 
T = t * G
D = T + B

# Bob scans...
t_prime = 
T_prime = t_prime * G
D_prime = T_prime + B

assert D_prime == D

### Spend and Scan Key

<p>Since Bob needs his private key <em>b</em> to check for incoming payments, this requires <em>b</em> to be exposed to an online device. To minimize the risks involved, Bob can instead publish an address of the form <em>(B<sub>scan</sub>, B<sub>spend</sub>)</em>. This allows Bob to keep <em>b<sub>spend</sub></em> in offline cold storage and perform the scanning with the public key <em>B<sub>spend</sub></em> and private key <em>b<sub>scan</sub></em>. Alice performs the tweak using both of Bob’s public keys in the following manner:</p>
<ul>
<li>Let <em>P<sub>0</sub> = HASH(outpoints_hash·a·B<sub>scan</sub> || 0)·G + B<sub>spend</sub></em></li>
</ul>
<p>Bob detects this payment by calculating <em>P<sub>0</sub> = HASH(outpoints_hash·b<sub>scan</sub>·A)·G + B<sub>spend</sub></em> with his online device and can spend from his cold storage signing device using <em>(HASH(outpoints_hash·b<sub>scan</sub>·A) + b<sub>spend</sub>) mod p</em> as the private key.</p>


In [ ]:
b_scan, B_scan = generate_bip340_key_pair()
b_spend, B_spend = generate_bip340_key_pair()

# doesn't have to be a taproot UTXO
txid1 = b'f4184fc596403b9d638783cf57adfe4c75c605f6356fbc91338530e9831e9e16'
vout1 = (0).to_bytes(4, 'little')

a1 = ECKey().generate()
A1 = a1.get_pubkey()

# taproot UTXO
txid2 = b'f4184fc596403b9d638783cf57adfe4c75c605f6356fbc91338530e9831e9e16'
vout2 = (0).to_bytes(4, 'little')

a2, A2 = generate_bip340_key_pair()

outpoint_hash = sha256(txid1 + vout1 + txid2 + vout2)

# sha256(outpoint_hash * sum of Alice's privkeys * Bob's scan public key)
t = 
T = t * G
# add to Bob's spend public key
D = 

# Bob scans..
t_prime = 
T_prime = t_prime * G
D_prime = 

assert D_prime == D

### Labels

<p>For a single silent payment address of the form <em>(B<sub>scan</sub>, B<sub>spend</sub>)</em>, Bob may wish to differentiate incoming payments by using labels. Naively, Bob could publish multiple silent payment addresses, but this would require him to scan for each one, which becomes prohibitively expensive. Instead, Bob can tweak his spend public key <em>B<sub>spend</sub></em> with an integer <em>m</em> in the following way:</p>
<ul>
<li>Let <em>B<sub>m</sub> = B<sub>spend</sub> + m·G</em></li>
<li>Publish <em>(B<sub>scan</sub>, B<sub>0</sub>)</em>, <em>(B<sub>scan</sub>, B<sub>1</sub>) …</em></li>
</ul>
<p>Alice performs the tweak same as before using one of the published <em>(B<sub>scan</sub>, B<sub>m</sub>)</em> pairs. Bob detects the labeled payment in the following manner:</p>
<ul>
<li>Let <em>P<sub>0</sub> = HASH(outpoints_hash·b<sub>scan</sub>·A || 0)·G + B<sub>spend</sub></em></li>
<li>Compute <em>P<sub>0m</sub> = P<sub>0</sub> + m·G</em> for each <em>m</em></li>
<li>For each <em>P<sub>0m</sub></em> in {<em>P<sub>00</sub> … P<sub>0m</sub></em>}, check if any are present in the transaction outputs</li>
</ul>

In [ ]:
b_scan, B_scan = generate_bip340_key_pair()
b_spend, B_spend = generate_bip340_key_pair()

# create some labels
labels = {
    'twitter': B_spend + ,
    'github': B_spend + ,
    'project a': B_spend + ,
    'project b': B_spend + ,
}

# doesn't have to be a taproot UTXO
txid1 = b'f4184fc596403b9d638783cf57adfe4c75c605f6356fbc91338530e9831e9e16'
vout1 = (0).to_bytes(4, 'little')

a1 = ECKey().generate()
A1 = a1.get_pubkey()

# taproot UTXO
txid2 = b'f4184fc596403b9d638783cf57adfe4c75c605f6356fbc91338530e9831e9e16'
vout2 = (0).to_bytes(4, 'little')

a2, A2 = generate_bip340_key_pair()

# Bob finds Alice's silent payment address through github and makes a payment
B_2 = labels['github']

outpoint_hash = sha256(txid1 + vout1 + txid2 + vout2)
t = 
T = t * G
D = 

# Bob scans..
t_prime = 
T_prime = t_prime * G

D_prime = None
for k,V in labels.items():
        
assert D_prime == D

## Putting it all together

### Simple wallet: generating silent payment keys

* Use BIP32 hardened derivation for the silent payment keys
* Generate a silent payment address


In [ ]:
class SPWallet:
    
    REASON = "1337"

    def __init__(self, seed):
        self.master = BIP32.from_seed(bytes.fromhex(seed))
        self.spend_path = 
        self.scan_path = 
        self.scan_privkey, self.scan_pubkey = self.convert_to_bip340_key_pair(
            self.get_scan_privkey()
        )
        self.spend_privkey, self.spend_pubkey = self.convert_to_bip340_key_pair(
            self.get_spend_privkey()
        )
        
    def convert_to_bip340_key_pair(self, seckey):
        d = seckey * G
        P = d.get_pubkey()
        if P.get_y() % 2 != 0:
            d.negate()
            P.negate()
        return d, P
        
    def get_scan_pubkey(self):
        return self.master.get_pubkey_from_path(self.scan_path)
    
    def get_scan_privkey(self):
        return self.master.get_privkey_from_path(self.scan_path)
    
    def get_spend_pubkey(self):
        return self.master.get_pubkey_from_path(self.spend_path)
    
    def get_spend_privkey(self):
        return self.master.get_privkey_from_path(self.spend_path)
    
    def get_silent_payment_address(self):
        pubkeys = 
        
        data = convertbits(pubkeys, 8, 5)
        return bech32_encode("sprt", data, Encoding.BECH32M)

### Scanning:  BIP158 block filters

* Get the tweak data per transaction (using `getsilentpaymentblockdata` rpc)
* Do the ECDH tweaks
* Check if any outputs exist in a block using BIP158 block filters

In [ ]:
class SPScanner:
    PUBKEY_BYTES = 33
    TRUNC_HASH_BYTES = 8
    WITNESS_VERSION_1 = '5120'
    
    def __init__(self, spend_pubkey, scan_privkey, rpc_client, start_height):

        self.spend_pubkey = spend_pubkey
        self.scan_privkey = scan_privkey
        self.start_from_height = start_height
        self.last_block_scanned = self.start_from_height - 1
        self.rpc = rpc_client
        
    def sha256_secp256k1_ecdh(self, x32, y32):
        
        version = (y32[31] & 0x01) | 0x02
        sha = hashlib.sha256()
        sha.update(bytes([version]))
        sha.update(bytes(x32))
        
        return sha.digest()
        
    def refresh(self, rpc_client):
        self.rpc = rpc_client
        
    def scan(self, start=None):
    
        if start:
            self.start_from_height = start
            
        # get the chain height
        stop = self.rpc.getblockchaininfo()['blocks']
        current_block = self.start_from_height
        while current_block <= stop:
            
            # get the silent payment tweak data
            res = 
            if res['total_tx'] == 0:
                # if there are no silent payment "eligible" txs, move on
                current_block += 1
                continue
            
            # get the tweak data 
            tweak_data = 
            
            # compute the tweaks
            outputs_to_check = 
            
            # check if any of the outputs exist in the block
            # if they do, they necessarily belong to us
            self.is_mine(outputs_to_check, res['block_hash'])
            current_block += 1
            
        self.start_from_height = current_block
        print("done scanning")
        
    def get_silent_payment_block_data(self, height):
            
        # get the block hash for a given height
        block_hash = 
        
        # get the silent payment block data
        data = 
        
        # add the block hash to the response (saves an rpc call later)
        data['block_hash'] = block_hash
        return data
        
    def parse_silent_payment_block_data(self, data):
        total_txs = data['total_tx']
        tx_data = data['data']
        txs = [
            tx_data[i:i + ?]
            for i in range(0, len(tx_data), ?
        ]
        tweaks = [
            (tx[:self.PUBKEY_BYTES*2], tx[self.PUBKEY_BYTES*2:]) for tx in txs
        ]
        assert len(tweaks) == total_txs
        return tweaks
    
    def compute_outputs(self, txs):
        potential_outputs = []
        for (sum_pubkeys, trunc_hash) in txs:
            # convert the hex pubkey into an ECPubKey
            I = 
            
            # hash the truncated hash
            outpoint_hash = 
            
            # compute the tweak
            ecdh = 
            
            # SHA256(outpoint_hash * a * B) - use the special hash function which implements secp256k1 ecdh
            shared_secret = self.sha256_secp256k1_ecdh(
                ecdh.get_x().to_bytes(32, 'big'),
                ecdh.get_y().to_bytes(32, 'big'),
            )
            
            # t * G + A
            pubkey = 
            
            # create a taproot scriptPubKey: 1 <pubkey>
            spk = 
            potential_outputs.append(spk)
            
        return potential_outputs
    
    def get_compact_block_filter(self, block_hash):
        res = self.rpc.getblockfilter(block_hash)
        
        # cheating here, because this is actually a compactSize field
        N = int(res['filter'][:2], 16)
        block_filter = res['filter'][2:]
        return {'size': N, 'filter': block_filter}
    
    def is_mine(self, script_pub_keys, block_hash):
        
        gcs_data = self.get_compact_block_filter(block_hash)
        if gcs_match_any(block_hash, script_pub_keys, gcs_data):
            block_data = self.rpc.getblock(block_hash, 2)
            for tx in block_data['tx']:
                for vout in tx['vout']:
                    if vout['scriptPubKey']['hex'] in script_pub_keys:
                        print(f"{tx['txid']} is mine!")
        
def rpc_client():
    return AuthServiceProxy("http://%s:%s@127.0.0.1:18443"%("silent", "payments"))

wallet = SPWallet(sha256(b'stuff n fluff').hex())
wallet.get_silent_payment_address()

In [ ]:
scanner = SPScanner(wallet.spend_pubkey, wallet.scan_privkey, rpc_client(), 315)
scanner.scan()